
I recently became interested in geohashing and wanted to develop an understanding of the algorithm with the aim of implementing it myself. I was surprised to find it to be quite simple and intuitive. In this article, I'll demonstrate how to generate geohashes for a given latitude and longitude and compare results against `pygeohash`, a Python library for all things geohash-related. I'll also walkthrough an approach that can be used to find neighboring cells of a given geohash, and render them on a map using folium. 

A geohash is a unique identifier of a specific region on the Earth. The basic idea is the Earth gets divided into rectangular regions of user-defined size, and each region is assigned a unique id which is called its geohash. For a given location on earth, the algorithm converts an arbitrary precision latitude and longitude into a string, the idea being that regions with a similar string prefix will be closer together. Conceptually, geohashing reduces proximity search to string prefix matching. As each character encodes additional precision, shared prefixes denote geographic proximity. But the converse is not true: Two points may be close in terms of relative proximity, but have no common geohash characters (think of two points on opposite sides of the prime meridian). 

Geohashes offer a level of anonymity by avoiding the need to share precise GPS coordinates. Instead, they represent a location within a specific bounding box at a chosen level of precision.

Before we begin, it is important to note that geohash libraries in the wild use a more efficient generating mechanism than what is presented here. My goal is to demonstrate the concept with maximum clarity as opposed to maximum efficiency. If you decide to use geohashing in a real-world application, use an existing library. 

<br>


### Algorithm

A geohash is a hierarchical spatial index: In order to represent a point, the world is recursively divided into smaller and smaller grid cells with each additional bit until the desired precision is reached. Functionally, the algorithm works by storing the intermediate results of two binary searches. In the resulting bit string, even-indexed bits represent longitude, while odd-indexed bits represent latitude. The user specifies a level of precision, usually between 1 and 12, and a geohash of that length is returned. The table below gives the dimensions of geohashes at each level of precision (taken from [here](https://www.movable-type.co.uk/scripts/geohash.html)):


```text
Precision          Dimension     
        1: 5,000km x 5,000km
        2:   1,250km x 625km
        3:     156km x 156km
        4:   31.9km x 19.5km
        5:   4.89km x 4.89km
        6:   1.22km x 0.61km
        7:       153m x 153m
        8:     38.2m x 19.1m
        9:     4.77m x 4.77m
       10:    1.19m x 0.596m 
       11:     149mm x 149mm
       12:   37.2mm x 18.6mm
``` 

The values in the table represent the maximum dimension at each level of precision: As locations move away from the equator, the dimensions of each cell get smaller. 

For a geohash of length 12, the coordinate can be found in a cell having dimension 37.2mm x 18.6mm, which is an impressive level of precision for a 12 character string! 

The geohash symbol map consists of 32 characters:

```
base32 = "0123456789bcdefghjkmnpqrstuvwxyz"
```

`base32` consists of digits 0 thru 9 plus all lowercase letters excluding `a, i, l, o`. As the search space is recursively partitioned, each group of 5 bits, when converted to a decimal integer, maps to one of the values in `base32`. 

I'll present the algorithm first then walkthrough it step-by-step. What follows is a basic implementation of geohashing, which takes as input the latitude and longitude for a point of interest and returns the geohash and bounding box to the specified level of precision:


In [1]:
"""
Very inefficient geohashing subroutine. For demonstration purposes only. 
"""

def get_geohash(lat, lon, precision=12):
    """
    Generate geohash of lat/lon at specified precision.

    Parameters
    ----------
    lat: float
        Latitude of point of interest.

    lon: float
        Longitude of point of interest

    precision: int
        Precision of geohash. Higher values result in 
        smaller bounding regions.

    Returns
    -------
    geohash, bbox as list
    """
    base32 = "0123456789bcdefghjkmnpqrstuvwxyz"
    bits = []

    min_lat, max_lat = -90, 90
    min_lon, max_lon = -180, 180

    for ii in range(5 * precision):

        if ii % 2 == 0:
            # Bisect longitude (E-W).
            mid_lon = (min_lon + max_lon) / 2
            if lon >= mid_lon:
                bits.append(1)
                min_lon = mid_lon
            else:
                bits.append(0)
                max_lon = mid_lon

        else:
            # Bisect latitude (N-S).
            mid_lat = (min_lat + max_lat) / 2
            if lat >= mid_lat:
                bits.append(1)
                min_lat = mid_lat
            else:
                bits.append(0)
                max_lat = mid_lat

    # Create single bit string from list of 0/1s. 
    bitstr = "".join([str(ii) for ii in bits])

    # Partition bitstr into groups of 5 bits. 
    quints = [bitstr[(5 * ii):(5 * (ii + 1))] for ii in range(precision)]

    # Convert binary digits to decimal digits to get indices into base32. 
    indices = [int(ii, base=2) for ii in quints]

    # Lookup characters associated with each index and concatenate.
    geohash = "".join([base32[ii] for ii in indices]) 

    # geohash bounding box is just the final, min_lat, min_lon, max_lat, max_lon.
    bbox = [min_lat, min_lon, max_lat, max_lon]

    return geohash, bbox


<br>


Within `get_geohash`, `base32` represents the 32 character map and `bits` gets set to an empty list to hold 0s and 1s. The initial minimum and maximum latitudes and longitudes define the bounds of the search space. 

The range of iteration is specified as `range(5 * precision)`. For a geohash having precision=6, the `bits` list will contain   5 * 6 = 30 values. We multiply by 5 since the `base32` character map has length 32, and 2^5 bit arrangements cover each of the 32 indices. 

If `ii` is even, we bisect longitude. If `lon >= mid_lon` we append 1 to `bits` and update `min_lat`. Otherwise we append 0 and update `max_lon`. If `ii` is odd, we execute the same logic but for latitude. In this way, the final bit string represents interleaved longitude and latitude bits. 

Once iteration completes, the individual elements of `bits` are concatenated into a single bit string `bitstr`. Next, we partition `bitstr` into groups of 5 bits (`quints`), which are converted to decimal integers, which are then used as indices to lookup elements from `base32`. These characters are concatenated into a single string representing the geohash. 


Let's use `get_geohash` to determine the level 6 geohash for The Mutual Group home office in West Des Moines, IA (41.58538, -93.71946). `bits` will consist of 30 1s and 0s:


```
[0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0]
```

Which get combined into a single bit string:

```
'010011111110011100100110100110'
```

Partitioned into groups of 5 bits:

```
['01001', '11111', '10011', '10010', '01101', '00110']
```
  
Converted to decimal integers:

```
[9, 31, 19, 18, 13, 6]
```

  
Which serve as indices into `base32`, whose corresponding elements and are concatenated into the geohash:

```
'9zmke6'
```

  
`get_geohash` also returns the bounding box of the precision = 6 geohash:

```
[41.583251953125, -93.724365234375, 41.5887451171875, -93.71337890625]
```

<br>

Let's compare the results of our implementation against pygeohash to ensure consistency:



In [2]:

import pygeohash

lat, lon, precision = 41.58538, -93.71946, 6
gh1 = get_geohash(lat, lon, precision)[0]
gh2 = pygeohash.encode(latitude=lat, longitude=lon, precision=precision)

print(f"get_geohash level-6 geohash : {gh1}")
print(f"pygeohash level-6 geohash   : {gh2}")


get_geohash level-6 geohash : 9zmke6
pygeohash level-6 geohash   : 9zmke6



<br>

We can visualize the precision = 6 geohash using folium:

In [3]:

import folium 
from folium.features import DivIcon


lat, lon, precision = 41.58538, -93.71946, 6

# Use our subroutine to get geohash and bounding box. 
geohash, bbox = get_geohash(lat, lon, precision=precision)

# Unpack bbox.
min_lat, min_lon, max_lat, max_lon = bbox

# Get mid_lat and mid_lon for geohash id placement. 
mid_lat = (min_lat + max_lat) / 2
mid_lon = (min_lon + max_lon) / 2

m = folium.Map(
    location=[lat, lon], 
    #width=900, 
    #height=600, 
    zoom_start=15, 
    zoom_control=True, 
    no_touch=True,
    tiles="OpenStreetMap"
    )

# precision = 6 geohash bounding box. 
folium.Rectangle(
    [(min_lat, min_lon), (max_lat, max_lon)], 
    fill_color="blue", fill_opacity=.15
    ).add_to(m)

# Red dot at Merchandise Mart. 
folium.CircleMarker(
    location=[lat, lon], radius=5, color="red", fill_color="red", 
    fill_opacity=1
    ).add_to(m)

# precision = 6 geohash id.
folium.map.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(100,50),
        html=f'<div style="font-size: 40pt">{geohash}</div>',
        )
    ).add_to(m)

m



<br>

### Identify Neighboring Cells

`get_geohash` can be used to help identify cells that neighbor a given 
geohash. Once the bounding box for the target geohash is known we simply increment 
those coordinates by a small amount, then lookup the geohash and bounding box 
associated with the new coordinate:

In [4]:

lat, lon, precision = 41.58538, -93.71946, 6

eps = 1e-10

# Center geohash id and bounding box.
geohash, bbox = get_geohash(lat, lon, precision=precision)

min_lat, min_lon, max_lat, max_lon = bbox

# Get geohash id and bounding box for Northwest cell.
gh_nw, bb_nw = get_geohash(max_lat + eps, min_lon - eps, precision=precision)

# Get geohash id and bounding box for Northeast cell.
gh_ne, bb_ne = get_geohash(max_lat + eps, max_lon + eps, precision=precision)

# Get geohash id and bounding box for Southeast cell.
gh_se, bb_se = get_geohash(min_lat - eps, max_lon + eps, precision=precision)

# Get geohash id and bounding box for Southwest cell.
gh_sw, bb_sw = get_geohash(min_lat - eps, min_lon - eps, precision=precision)



<br>

Using the results from the NW, NE, SE and SW directions, we obtain the geohashes
in the N, S, E and W directions:

In [5]:

# For N, get mid point between NE and NW cells.
min_lat_nw, min_lon_nw, max_lat_nw, max_lon_nw = bb_nw
min_lat_ne, min_lon_ne, max_lat_ne, max_lon_ne = bb_ne
n_lat = (min_lat_nw + max_lat_nw) / 2
n_lon = (min_lon_nw + max_lon_ne) / 2

# For S, get mid point between SE and SW cells.
min_lat_sw, min_lon_sw, max_lat_sw, max_lon_sw = bb_sw
min_lat_se, min_lon_se, max_lat_se, max_lon_se = bb_se
s_lat = (min_lat_sw + max_lat_sw) / 2
s_lon = (min_lon_sw + max_lon_se) / 2

# For E, get mid point between SE and NE cells.
min_lat_ne, min_lon_ne, max_lat_ne, max_lon_ne = bb_ne
min_lat_se, min_lon_se, max_lat_se, max_lon_se = bb_se
e_lat = (max_lat_ne + min_lat_se) / 2
e_lon = (min_lon_se + max_lon_se) / 2

# For W, get mid point between SW and NW cells.
min_lat_nw, min_lon_nw, max_lat_nw, max_lon_nw = bb_nw
min_lat_sw, min_lon_sw, max_lat_sw, max_lon_sw = bb_sw
w_lat = (max_lat_nw + min_lat_sw) / 2
w_lon = (min_lon_sw + max_lon_sw) / 2

![image01](image01.png)



<br>

This logic can be encapsulated within a function identified as `get_neighbors`:



In [6]:


def get_neighbors(lat, lon, precision=12):
    """
    Find 8 adjacent neighbors to geohash associated
    with (lat, lon) at specified precision.

    Parameters
    ----------
    lat: float
        Latitude of point of interest.

    lon: float
        Longitude of point of interest

    precision: int
        Precision of geohash. Higher values result in 
        smaller bounding regions.

    Returns
    -------
    dict
    """
    dneighbors = {}
    eps = 1e-10

    # Center geohash id and bounding box.
    geohash, bbox = get_geohash(lat, lon, precision=precision)
    min_lat, min_lon, max_lat, max_lon = bbox

    dneighbors["center"] = {"geohash": geohash, "bbox": bbox}

    dcorners = {
        "nw": {"lat": max_lat + eps, "lon": min_lon - eps},
        "ne": {"lat": max_lat + eps, "lon": max_lon + eps},
        "sw": {"lat": min_lat - eps, "lon": min_lon - eps},
        "se": {"lat": min_lat - eps, "lon": max_lon + eps},
        }

    for kk in dcorners:
        lat_, lon_ = dcorners[kk]["lat"], dcorners[kk]["lon"]
        gh, bb = get_geohash(lat_, lon_, precision=precision)
        dneighbors[kk] = {"geohash": gh, "bbox": bb}

    # Find geohash for N, S, E and W directions.
    min_lat_nw, min_lon_nw, max_lat_nw, max_lon_nw = dneighbors["nw"]["bbox"]
    min_lat_ne, min_lon_ne, max_lat_ne, max_lon_ne = dneighbors["ne"]["bbox"]
    n_lat = (min_lat_nw + max_lat_nw) / 2
    n_lon = (min_lon_nw + max_lon_ne) / 2
    n_gh, n_bb = get_geohash(n_lat, n_lon, precision=precision)
    dneighbors["n"] = {"geohash": n_gh, "bbox": n_bb}

    min_lat_sw, min_lon_sw, max_lat_sw, max_lon_sw = dneighbors["sw"]["bbox"]
    min_lat_se, min_lon_se, max_lat_se, max_lon_se = dneighbors["se"]["bbox"]
    s_lat = (min_lat_sw + max_lat_sw) / 2
    s_lon = (min_lon_sw + max_lon_se) / 2
    s_gh, s_bb = get_geohash(s_lat, s_lon, precision=precision)
    dneighbors["s"] = {"geohash": s_gh, "bbox": s_bb}

    min_lat_ne, min_lon_ne, max_lat_ne, max_lon_ne = dneighbors["ne"]["bbox"]
    min_lat_se, min_lon_se, max_lat_se, max_lon_se = dneighbors["se"]["bbox"]
    e_lat = (max_lat_ne + min_lat_se) / 2
    e_lon = (min_lon_se + max_lon_se) / 2
    e_gh, e_bb = get_geohash(e_lat, e_lon, precision=precision)
    dneighbors["e"] = {"geohash": e_gh, "bbox": e_bb}

    min_lat_nw, min_lon_nw, max_lat_nw, max_lon_nw = dneighbors["nw"]["bbox"]
    min_lat_sw, min_lon_sw, max_lat_sw, max_lon_sw = dneighbors["sw"]["bbox"]
    w_lat = (max_lat_nw + min_lat_sw) / 2
    w_lon = (min_lon_sw + max_lon_sw) / 2
    w_gh, w_bb = get_geohash(w_lat, w_lon, precision=precision)
    dneighbors["w"] = {"geohash": w_gh, "bbox": w_bb}

    return dneighbors


Running `get_neighbors` and iterating over the results yields:

In [7]:

lat, lon, precision = 41.58538, -93.71946, 6

dn = get_neighbors(lat, lon, precision=precision)

for kk in dn:
    gh, bb = dn[kk]["geohash"], dn[kk]["bbox"]
    print(f"{[kk]}: geohash={gh}, bbox={bb}")
    

['center']: geohash=9zmke6, bbox=[41.583251953125, -93.724365234375, 41.5887451171875, -93.71337890625]
['nw']: geohash=9zmke5, bbox=[41.5887451171875, -93.7353515625, 41.59423828125, -93.724365234375]
['ne']: geohash=9zmkee, bbox=[41.5887451171875, -93.71337890625, 41.59423828125, -93.702392578125]
['sw']: geohash=9zmke1, bbox=[41.5777587890625, -93.7353515625, 41.583251953125, -93.724365234375]
['se']: geohash=9zmke9, bbox=[41.5777587890625, -93.71337890625, 41.583251953125, -93.702392578125]
['n']: geohash=9zmke7, bbox=[41.5887451171875, -93.724365234375, 41.59423828125, -93.71337890625]
['s']: geohash=9zmke3, bbox=[41.5777587890625, -93.724365234375, 41.583251953125, -93.71337890625]
['e']: geohash=9zmked, bbox=[41.583251953125, -93.71337890625, 41.5887451171875, -93.702392578125]
['w']: geohash=9zmke4, bbox=[41.583251953125, -93.7353515625, 41.5887451171875, -93.724365234375]


<br>

Finally, we can visualize a geohash with its 8 surrounding neighbors:

In [8]:
list(dn.keys())[:5]

['center', 'nw', 'ne', 'sw', 'se']

In [9]:

import folium 
from folium.features import DivIcon
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


lat, lon, precision = 41.58538, -93.71946, 6

# Get  geohashes and bounding boxes. 
dn = get_neighbors(lat, lon, precision=6)

# Separate color for each cell.
fcolors = mpl.colormaps["gist_rainbow"]
colors_rgba = [fcolors(ii) for ii in np.linspace(0, 1, len(dn))]
colors_hex = [mpl.colors.to_hex(ii, keep_alpha=False) for ii in colors_rgba]


m = folium.Map(
    location=[lat, lon], 
    # width=900, 
    # height=600, 
    zoom_start=15, 
    zoom_control=True, 
    no_touch=True,
    tiles="OpenStreetMap"
    )

for ii, kk in enumerate(dn.keys()):
    color = colors_hex[ii]
    geohash, bbox = dn[kk]["geohash"], dn[kk]["bbox"]
    min_lat, min_lon, max_lat, max_lon = bbox
    mid_lat = (min_lat + max_lat) / 2
    mid_lon = (min_lon + max_lon) / 2
    
    # geohash bounding box. 
    folium.Rectangle(
        [(min_lat, min_lon), (max_lat, max_lon)], 
        fill_color=color, fill_opacity=.25
        ).add_to(m)
    
    folium.map.Marker(
        [mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(250,36),
            icon_anchor=(75,25),
            html=f'<div style="font-size: 30pt">{geohash}</div>',
            )
        ).add_to(m)

m

<br>

As we've demonstrated, geohashes are useful because they compactly encode geographic coordinates into strings, enabling efficient location-based indexing and searches. This is useful for data aggregation and protects privacy by providing only approximate locations without revealing exact coordinates. 